# 1 Introduction

Welcome back!

------
In our last lesson, we

* read  NYC ER data into a Pandas dataframe
* made simple plots in Altair

------
Today we will,
* read in pandemic data from the Johns Hopkins repository
* select columns and rows
* group the data
* create a new column calculated from others
* sort the data 
* select the top 20 countries according to some measure

Your homework (in another notebook) will be to apply these techniques to  US states and Chinese provinces.

Obviously, there is more one could do with this data.  Baby steps.

Naturally, you need to make a copy of this notebook in your directory first.

# 2 Imports

In [ ]:
import pandas as pd


# 3 Daily COVID data from `coronavirus.jhu.edu`

## 3.1 Reading in the data
You are probably familiar with the Johns Hopkins corona virus map which can be found here: https://coronavirus.jhu.edu/map.html

The JHU folks kindly share the data they aggregate for the maps in a github repo: https://github.com/CSSEGISandData/COVID-19.

We can read this data into a Pandas dataframe if we can find the right URL.  Basically:

* find the github webpage for the data file
* click on the `raw` button
* copy the URL for the `raw` page
* drop that URL into a Pandas `read_csv()` statement

Let's do it!  I already found the `raw` URL for you.

In the line below, we read in the daily report for March 22, 2021.

In [ ]:
covid = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/12-02-2022.csv")

In [ ]:
covid.head()

I wonder how large this data frame is.  We can use the `.shape` attribute to find out. Remember that attributes don't have `()`, but methods do.

In [ ]:
covid.shape

Recall that `dataframe.columns` is also an attribute of every dataframe.  It returns an index (like a list) of column names.

In [ ]:
covid.columns

So what so some of these columns mean?  The explanations appear below the map on the Johns Hopkins site [here](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data#field-description).  I copied them over for you.

* **FIPS:** US only. Federal Information Processing Standards code that uniquely identifies counties within the USA.
* **Admin2:** County name. US only.
* **Province_State:** Province, state or dependency name.
* **Country_Region:** Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* **Last Update:** `MM/DD/YYYY HH:mm:ss` (24 hour format, in UTC).
* **Lat and Long_:** Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* **Confirmed:** Counts include `confirmed` and `probable` (where reported).
* **Deaths:** Counts include `confirmed` and `probable` (where reported).
* **Recovered:** Recovered cases are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from COVID Tracking Project.
* **Active:** `Active cases` = `total cases` - `total recovered` - `total deaths`.
* **Incident_Rate:** `Incidence Rate` = `cases` per `100,000 persons`.
* **Case_Fatality_Ratio (%):** `Case-Fatality Ratio (%)` = `Number recorded deaths` / `Number cases`.

Later on we'll be looking at the `Case_Fatality_Ratio`.

To scroll through all 3981 rows, execute the following cell.  I think that some of you found the previous assignment challenging at first because you couldn't picture what was actually in the data.  This would help.

What do you notice about the values?  Is there a row for each country?

In [ ]:
pd.set_option('display.max_rows', None)  
covid

Some lines represent whole countries.  Other countries are broken up into provinces/states.  The United States data is the most granular--it goes all the way down to county.

What in the world is `NaN`?  In Pandas, this means 'not a number'.  It appears in a numerical columns for entries that are ontain something other than numbers.  It also appears for blank values in other columns.

Presently the `FIPS` county code was interpreted as a `float64` (ie, it looks like a decimal). You can scroll down again and see that if you missed it.  We'd rather `FIPS` be interpreted as a string, so that leading zeros don't disappear (that's a typical problem with those codes).  It would be a distraction to deal with this right now. If you insist...see discussion [here](https://stackoverflow.com/questions/13250046/how-to-keep-leading-zeros-in-a-column-when-reading-csv-with-pandas).  The proposed simple fixes don't work.  Just make a mental note that this can be an issue.

## 3.2 Select 

Next we review a way to select rows of a dataframe and introduce a way to select columns.  A more comprehensive overview of dataframe selection methods can be found [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html#Data-Selection-in-DataFrame).  We are going to start with techniques using `[]` (index) notation.

### 3.2.1 Select columns

You can select a single column using `dataframe['column_name']`.  For example:

In [ ]:
covid['Country_Region']

Btw, by scrolling through the output, you can see clearly that some countries, eg Austria, are represented by one row, but others, eg Australia, are represented by several.  Another useful method you can apply to a column is `value_counts()`, which returns all of the values that occur in a column along with the number of occurences.

In [ ]:
covid['Country_Region'].value_counts()

Finally, you can select multiple columns by passing a list of names rather than just a single name:  `dataframe[['column_A','column_B','column_C']]`.

### 3.2.2 Select rows

You can select rows that satisfy a condition using `dataframe[condition]`.  For example, to select all of the rows of the `covid` dataframe for which `Country_Region` is equal to `Denmark`, you could use the following.  The condition is `covid['Country_Region]=='Denmark'`.  Notice that the left side of that conditional is basically just a column of the dataframe (which is called a Pandas *series*).  

In [ ]:
covid[covid['Country_Region']=='Denmark']

## 3.3 Sort 

Pandas dataframes have a `sort_values()` method.  Its documentation is [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html).

The following sorts the dataframe
* by the values in the `Deaths` column
* since `ascending=False`, it sorts them in descending order (ie, large to small)
* if, after `ascending=False`, we added  `inplace=True`, it would replace our old version of the dataframe with this newly sorted version.  As it stands now, the output will be sorted but the `covid` dataframe itself will remain unsorted. 

You can also see in the documentation how you might sort multiple columns at a time.

In [ ]:
covid.sort_values('Deaths',ascending=False).head(10)

Did you catch the structure of the previous line of code?  `dataframe.sort_values().head()`.  The two methods `sort_values()` and `head()` are said to have been *chained* together.  They are executed in left to right order.

This was sort of a dumb thing to do.  We are producing a ranking which compares countries, provinces, and counties.  Sheesh.  Apples to apples and all that.  

The natural question that arises is:  what would a ranking of countries look like?  Is it easy to produce from this data?  Yes!

## 3.4 `Groupby`

Pandas dataframes also have a powerful `groupby()` method.  It's explained in great detail [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html#GroupBy:-Split,-Apply,-Combine).  We will circle back to this at a later date.  Today, we're going to keep it simple.

What this next cell does is:
* group the rows with the same `Country_Region` together
* select only the columns `Confirmed`, `Deaths`, and `Recovered`
* `sum()` the data in the columns `Confirmed`, `Deaths`, and `Recovered`

Notice that the `groupby()`, column selection, and `sum()` methods are more or less chained.  Btw, people tend to use the term 'chaining' with methods and not with selection using square brackets.

In [ ]:
covid_country = covid.groupby("Country_Region")[['Confirmed','Deaths','Recovered']].sum()
covid_country.head()   #[:5] does the same thing as .head()

In [ ]:
covid_country.shape

Why only three columns in the shape?  Because the `Country_Region` is just an index!

To find the top 10 countries by the number of cases we could just

In [ ]:
covid_country.sort_values('Confirmed',ascending=False)[:10]

-------
**Note:**  You probably should ignore this now but...we could have reordered the `groupby()`, column selection, `sum()` in the following way

In [ ]:
covid[['Country_Region','Confirmed','Deaths','Recovered']].groupby("Country_Region").sum()

You get the same result as with the `groupby()` we used. I have only seen people order these in the way we did above.  It does save you the trouble of typing `Country_Region` twice!

## 3.5 Create a new column

We couldn't just sum the `Case-Fatality-Rate` (fractions don't work that way!), but once we sum the other columns, we can calculate it anew from those sums. 

Let's use the following formula:

In [ ]:
covid_country['Case_Fatality_Ratio']=round(covid_country.Deaths/covid_country.Confirmed*100,1)

Ok, that created a new column.  Now let's look at the new version of the dataframe.

In [ ]:
covid_country

## 3.6 The finale: Top 20 countries by `Case_Fatality_Ratio`

Of course, we could look at the top twenty countries according to `Case_Fatality_Ratio` using:

In [ ]:
covid_country.sort_values('Case_Fatality_Ratio',ascending=False)[:20]